In [5]:
class Processus:
    """Classe définissant un processus"""
    def __init__(self, pid, duree):
        self.pid = pid
        self._duree = duree  # ne pas modifier cet attribut

    def execute(self, quantum):
        """Diminue la durée d'exécution restante de quantum. Bloque à 0"""
        self._duree = max(0, self._duree - quantum)

    def est_fini(self):
        """Renvoie le booléen répondant à la question 'Le processus est-il terminé ?'"""
        return self._duree == 0

    def __repr__(self):
        """Affichage"""
        return f"(PID={self.pid} ; durée={self._duree})"

In [18]:
class File():
    def __init__(self):
        self.file = []

    def est_vide(self):
        return len(self.file) == 0

    def enfile(self, elt):
        self.file.append(elt)

    def defile(self):
        assert not self.est_vide(), "une file vide ne peut être défilée"
        return self.file.pop(0)

    def tete(self):
        return self.file[0]

    def __repr__(self):
        return str(self.file)

Processus et tourniquet:

In [7]:
def tourniquet(file, quantum):
    ordre = []
    while not file.est_vide():
        processus = file.defile()
        ordre.append(processus.pid)
        processus.execute(quantum)
        if not processus.est_fini():
            file.enfile(processus)
    return ordre

In [8]:
f = File()

In [9]:
f.enfile(Processus(0, 5))

In [10]:
f.enfile(Processus(1, 2))

In [11]:
print(tourniquet(f, 1))

[0, 1, 0, 1, 0, 0, 0]


Etat d'un processus

In [12]:
class Processus():

    def __init__(self, nom, duree, arrivee=0):
        self.etat = "non créé"
        self.nom = nom
        self.duree = duree
        self.arrivee = arrivee
        self.temps_aloue = 0
        
    def elit(self, quantum):
        self.temps_aloue = quantum
        self.etat = "élu"
        
    def execute(self):
        self.duree = self.duree - 1
        self.temps_aloue = self.temps_aloue - 1
        if self.duree == 0:
            self.etat = "terminé"
        elif self.temps_aloue == 0:
            self.etat = "prêt"
            
    def est_pret(self, horloge):
        if self.arrivee == horloge:
            self.etat = "prêt"
        return True
    
    def bloque(self):
        self.etat = "bloqué"
        
    def debloque(self):
        self.etat = "prêt"          

    def __repr__(self):
        return str({ "etat": self.etat, "nom":self.nom, "arrivee" :self.arrivee, "duree":self.duree})

Ordonnance de processus

In [33]:
QUANTUM = 2
def temps_execution(arrivees, quantum):
    tps_execution = {}
    temps_horloge = 0
    file_attente = File()
    terminee = False
    processus_courant = None
    while not (arrivees.est_vide() and file_attente.est_vide() and processus_courant == None):
        
        if  not arrivees.est_vide() and arrivees.tete().est_pret(temps_horloge):
            file_attente.enfile(arrivees.defile())
            
        if processus_courant != None and processus_courant.etat == "prêt":
                file_attente.enfile(processus_courant)
                processus_courant = None
                    
        if processus_courant == None and not file_attente.est_vide():
            processus_courant = file_attente.defile()
            processus_courant.elit(quantum)
            debut_quantun = temps_horloge 
        
        temps_horloge = temps_horloge + 1
        if processus_courant != None:
            processus_courant.execute()
            if processus_courant.etat != "élu":
                if processus_courant.nom in tps_execution:
                    tps_execution[processus_courant.nom].append((debut_quantun,temps_horloge))
                else:
                    tps_execution[processus_courant.nom] = [(debut_quantun,temps_horloge)]
                if processus_courant.etat == "terminé":
                    processus_courant = None

        
    return tps_execution

In [34]:
QUANTUM = 2

In [35]:
arrivees = File()

In [36]:
arrivees.enfile(Processus("P1", arrivee = 0, duree = 1))

In [39]:
temps_execution(arrivees, QUANTUM)

{'P1': [(0, 2), (2, 3)]}

In [38]:
arrivees = File()
arrivees.enfile(Processus("P1", arrivee = 0, duree = 3))